In [16]:
import argparse
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import json
import time
import torch.nn.functional as F
import sys
import cnn_drone_net_utils
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from PIL import Image

In [2]:
DATA_PATH = "./data"
OUT_PATH = "./output"
DATA_UAV = "UAV_Dataset"
DATA_UAV_COMPRESSED_FILENAME = f"{DATA_UAV}.zip"
DATA_BLOB_UAV_COMPRESSED = "https://nlpdatastorage.blob.core.windows.net/cnndata/UAV_Dataset.zip"

In [3]:
os.makedirs(OUT_PATH, exist_ok=True)
os.makedirs(DATA_PATH, exist_ok=True)

In [4]:
download_data =  DATA_UAV_COMPRESSED_FILENAME not in os.listdir(DATA_PATH) 
zipped = f'{DATA_PATH}/{DATA_UAV_COMPRESSED_FILENAME}'
if(download_data):
    print(f"Download data from {DATA_BLOB_UAV_COMPRESSED} and saving in {zipped}")
    cnn_drone_net_utils.download_data(DATA_BLOB_UAV_COMPRESSED, out_path=zipped)

print(f"Unzipping {zipped} to directory {DATA_PATH} and created folders: {', '.join(os.listdir(f'{DATA_PATH}/{DATA_UAV}'))}")
cnn_drone_net_utils.unzip_file(zipped, DATA_PATH)

Unzipping ./data/UAV_Dataset.zip to directory ./data and created folders: FlightZone, NonFlightZone


In [21]:
##### Hyper Parameters #####

lr = 0.003
batch_size = 64
dropout = 0.2

In [13]:
data_dir = f'{DATA_PATH}/{DATA_UAV}'
trainloader, testloader = cnn_drone_net_utils.load_split_train_test(data_dir, .2, batch_size = batch_size)
print(f'Initialized data loaders, classes: {trainloader.dataset.classes}')

Initialized data loaders, classes: ['FlightZone', 'NonFlightZone']


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
print(f'Loaded pretrained model: {models.resnet50.__name__}')

Loaded pretrained model: resnet50


In [22]:
print("Freezing the pretrained model's parameters")
for param in model.parameters():
    param.requires_grad = False
    
print("Initialzing the last fully connected layer of the pretrained model")
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(dropout),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=lr)
model.to(device)

Freezing the pretrained model's parameters


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
##### Train #####

epochs = 1
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device),
                                      labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = 
                        top_class == labels.view(*top_class.shape)
                    accuracy +=
                    torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
            
torch.save(model, f'{OUT_PATH}/model.pth')

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()